### Content-based Movie Rating Estimation

**import required packages**

In [1]:
%run liblecture.py

import math
import numpy as np
from numpy import linalg as LA
import pandas as pd

### Movies Weight Matrix on Genres

Read movie metadata from a csv file.

In [2]:
movies = pd.read_csv('/content/movies_w_imgurl.csv')
movies.head()

,movieId,imdbId,title,genres,imgurl
0,1,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,https://images-na.ssl-images-amazon.com/images...
1,2,113497,Jumanji (1995),Adventure|Children|Fantasy,https://images-na.ssl-images-amazon.com/images...
2,3,113228,Grumpier Old Men (1995),Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,4,114885,Waiting to Exhale (1995),Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,5,113041,Father of the Bride Part II (1995),Comedy,https://images-na.ssl-images-amazon.com/images...


Split genres and stack genres into one column

In [3]:
movieGenres = pd.DataFrame(data=movies['genres'].str.split('|').apply(pd.Series,1).stack(), columns=['genre'])
movieGenres.index = movieGenres.index.droplevel(1)

In [4]:
movieGenres

,genre
0,Adventure
0,Animation
0,Children
0,Comedy
0,Fantasy
...,...
9121,Fantasy
9121,Sci-Fi
9122,Documentary
9123,Comedy


Count movies that have each genre and then compute IDF of genres

In [5]:
genres = pd.DataFrame(data=movieGenres.groupby('genre')['genre'].count())
genres.columns = ['movieCount']

totalitems = movies.shape[0]

genres['idf'] = genres['movieCount'].apply(lambda x: math.log10(totalitems/x))

genres.head()

,movieCount,idf
genre,,
(no genres listed),18,2.70
Action,1545,0.77
Adventure,1117,0.91
Animation,447,1.31
Children,583,1.19


Join genre's IDF to movie genre DataFrame

In [6]:
movieGenreWeights = movieGenres.join(genres['idf'], on='genre')
movieGenreWeights

,genre,idf
0,Adventure,0.91
0,Animation,1.31
0,Children,1.19
0,Comedy,0.44
0,Fantasy,1.14
...,...,...
9121,Fantasy,1.14
9121,Sci-Fi,1.06
9122,Documentary,1.27
9123,Comedy,0.44


In [7]:
movieWeights = movies[['movieId']]

for genre in genres.index:
    movieGenreIdf = movieGenreWeights[movieGenreWeights['genre'] == genre][['idf']]
    movieGenreIdf = movieGenreIdf.rename(columns = {'idf':genre})
    movieWeights = movieWeights.join(movieGenreIdf)

movieWeights.fillna(0, inplace= True)

### Movie-Movie Cosine Similarity Matrix
Compute $l_2$-norm of movies.

In [8]:
movieNorms = pd.DataFrame(data = LA.norm(movieWeights.iloc[:,1:].values, ord=2, axis=1), index = movieWeights.index,columns=['norm2'] )
movieNorms

,norm2
0,2.34
1,1.89
2,0.89
3,0.94
4,0.44
...,...
9120,1.24
9121,1.97
9122,1.27
9123,0.44


Normalize movie vector so that similarity can be computed simply by inner product between vectors.

$$ cosine(u, v)=\frac{\sum_{\forall i}{u_i v_i}}{||u||_2||v||_2}=\sum_{\forall i}{\frac{u_i v_i}{||u||_2||v||_2}}=\sum_{\forall i}{\frac{u_i}{||u||_2}\frac{v_i}{||v||_2}}=u'\cdot v'$$

In [9]:
normalizedMovieWeights = movieWeights.iloc[:, 1:].divide(movieNorms['norm2'], axis=0)

#소수점 2자리만 보여줘
np.set_printoptions(precision=2)
pd.set_option('display.precision', 2)

normalizedMovieWeights

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.0,0.00,0.39,0.56,0.51,0.19,0.0,0.0,0.00,0.49,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
1,0.0,0.00,0.48,0.00,0.63,0.00,0.0,0.0,0.00,0.61,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
2,0.0,0.00,0.00,0.00,0.00,0.50,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.87,0.00,0.0,0.0,0.0
3,0.0,0.00,0.00,0.00,0.00,0.47,0.0,0.0,0.34,0.00,0.0,0.0,0.0,0.0,0.0,0.82,0.00,0.0,0.0,0.0
4,0.0,0.00,0.00,0.00,0.00,1.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9120,0.0,0.00,0.74,0.00,0.00,0.00,0.0,0.0,0.26,0.00,0.0,0.0,0.0,0.0,0.0,0.62,0.00,0.0,0.0,0.0
9121,0.0,0.39,0.46,0.00,0.00,0.00,0.0,0.0,0.00,0.58,0.0,0.0,0.0,0.0,0.0,0.00,0.54,0.0,0.0,0.0
9122,0.0,0.00,0.00,0.00,0.00,0.00,0.0,1.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0
9123,0.0,0.00,0.00,0.00,0.00,1.00,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0


Create item-item similarity matrix

In [10]:
sims = pd.DataFrame(data=np.matmul(normalizedMovieWeights, normalizedMovieWeights.T))

sims.index = movieWeights['movieId']
sims.columns = movieWeights['movieId']

sims

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  """Entry point for launching an IPython kernel.


movieId,1,2,3,4,5,6,7,8,9,10,...,161830,161918,161944,162376,162542,162672,163056,163949,164977,164979
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.00,0.81,0.09,0.09,0.19,0.00,0.09,0.64,0.00,0.25,...,0.00,0.19,0.00,0.00,0.00,0.29,0.47,0.0,0.19,0.0
2,0.81,1.00,0.00,0.00,0.00,0.00,0.00,0.80,0.00,0.32,...,0.00,0.23,0.00,0.00,0.00,0.36,0.58,0.0,0.00,0.0
3,0.09,0.00,1.00,0.94,0.50,0.00,1.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.63,0.54,0.00,0.0,0.50,0.0
4,0.09,0.00,0.94,1.00,0.47,0.00,0.94,0.00,0.00,0.00,...,0.08,0.00,0.34,0.34,0.60,0.60,0.00,0.0,0.47,0.0
5,0.19,0.00,0.50,0.47,1.00,0.00,0.50,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,1.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162672,0.29,0.36,0.54,0.60,0.00,0.00,0.54,0.45,0.00,0.48,...,0.06,0.36,0.26,0.26,0.46,1.00,0.34,0.0,0.00,0.0
163056,0.47,0.58,0.00,0.00,0.00,0.22,0.00,0.28,0.39,0.52,...,0.00,0.69,0.00,0.00,0.00,0.34,1.00,0.0,0.00,0.0
163949,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.0,0.00,1.0


## Recommend Movies based on Predicted Ratings

Read ratings as train and test datasets

In [11]:
#Read rating data

ratings = pd.read_csv('/content/ratings-9_1.csv')

train = ratings[ratings['type'] =='train'][['userId', 'movieId', 'rating']]
test = ratings[ratings['type'] =='test'][['userId', 'movieId', 'rating']]

Set test user ID

In [12]:
userId = 33

Check top rated movies of the test user

In [14]:
userRatings = train[train['userId'] == userId][['movieId','rating']]

topRatings = userRatings.sort_values(by='rating', ascending=False).head(20)

topRatings

displayMovies(movies, topRatings['movieId'].values, topRatings['rating'].values)

Predict item ratings for the test usres

In [15]:
recSimSums = sims.loc[userRatings['movieId'].values, :].sum().values

recSimSums = recSimSums + 1

recWeightedRatingSums = np.matmul(sims.loc[userRatings['movieId'].values,:].T.values, userRatings['rating'].values)

recitemRatings = pd.DataFrame(data = np.divide(recWeightedRatingSums, recSimSums), index=sims.index)

recitemRatings.columns = ['pred']

recitemRatings

,pred
movieId,
1,2.99
2,2.72
3,3.21
4,3.22
5,3.22
...,...
162672,3.07
163056,2.75
163949,2.67


Check recommended items

In [16]:
top30Movies = recitemRatings.sort_values(by='pred', ascending=False).head(30)

displayMovies(movies, top30Movies.index, top30Movies['pred'].values)

Compute MAE and RMSE for the test user 

In [18]:
userTestRatings = pd.DataFrame(data=test[test['userId'] == userId])

temp = userTestRatings.join(recitemRatings.loc[userTestRatings['movieId']], on='movieId')

mae = getMAE(temp['rating'], temp['pred'])
rmse = getRMSE(temp['rating'], temp['pred'])

print(f"MAE : {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

MAE : 0.9682
RMSE: 1.1347
